## Using llm inference to extract metadata

In [1]:
from metadata_extraction.run_llm_pipeline import Extraction

if "key error: Data" comes up, update the openai api key

In [2]:
import os 
os.environ['OPENAI_API_KEY'] = '########'

The Extraction Class takes in a fewshot config file and a few examples in the g325a file texts and returns extract metadata from a new document by learning from the examples

In [4]:
ext_class = Extraction("./metadata_extraction/configs/fewshot_v2.cfg", "./metadata_extraction/configs/examples_g325a_2.yml")
ext_class_certnat = Extraction("./metadata_extraction/configs/fewshot_certnat.cfg", "./metadata_extraction/configs/examples_certnat.yml") # Run this to init extractor that extracts from Certificate of Naturalization


In [11]:
ext_class.extract_metadata("""
FORM G-325 A REV. 4-1-67 Screened by 9/18/2023 FORM APPROVED T/C 24QUDGET BUREAU NO. 43-R436 BIOGRAPHIC UNITED STATES DEPARTMENT OF JUSTICE JAN 15 1969 INFORMATION Immigration and Naturalization Service (FAMILY NAME) (FIRST NAME) (MIDDLE NAME) MALE BIRTHDATE(MO-DAY-YR.) NATIONALITY DATE ALIEN REGISTRATION NO. WONG, BEN DONG FEMALE 1/22/1814 CHINESE (IF ANY A12 250 430 ALL OTHER NAMES USED CITY AND COUNTRY OF BIRTH SOCIAL SECURITY NO. (IF ANY) WONG, GUCK ON CANTON , CHINA 553-36-2579 FAMILY NAME FIRST NAME DATE, CITY AND COUNTRY OF BIRTH (IF KNOWN) CITY AND COUNTRY OF RESIDENCE FATHER WONG WEE DON DIED MOTHER (MAIDEN NAME) TOM GIM YEP - CANADA (RESIDENCE) was SPOUSE (IF NONE, so STATE) FAMILY NAME FIRST NAME BIRTHDATE CITY & COUNTRY OF BIRTH DATE OF MARRIAGE PLACE OF MARRIAGE (FOR WIFE, GIVE MAIDEN NAME) FONG SUEY GiN 7/28/15 CANTON, CHINA, CHINA FORMER SPOUSES (IF NONE, so STATE) FAMILY NAME (FOR WIFE. GIVE MAIDEN NAME) FIRST NAME BIRTHDATE DATE & PLACE OF MARRIAGE DATE AND PLACE OF TERMINATION OF MARRIAGE NONE APPLICANT'S RESIDENCE LAST FIVE YEARS. LIST PRESENT ADDRESS FIRST. FROM TO STREET AND NUMBER CITY PROVINCE OR STATE COUNTRY MONTH YEAR MONTH YEAR 1416 310 AVENUE LOS ANGELES CALIF. U.S.A. SEPT. 52 PRESENT TIME LAST FOREIGN RESIDENCE OF MORE THAN ONE YEAR IF NOT SHOWN ABOVE. (INCLUDE ALL INFORMATION REQUESTED ABOVE.) APPLICANT'S EMPLOYMENT LAST FIVE YEARS. (IF NONE, so STATE) LIST PRESENT EMPLOYMENT FIRST. FROM TO FULL NAME AND ADDRESS OF EMPLOYER OCCUPATION MONTH YEAR MONTH YEAR SELF-EMPLOYED 1800 W-8 ST. LAUNDRY JULY 41 PRESENT TIME LAST OCCUPATION ABROAD IF NOT SHOWN ABOVE. (INCLUDE ALL INFORMATION REQUESTED ABOVE.) THIS FORM IS SUBMITTED IN CONNECTION WITH APPLICATION FOR: IF YOUR NATIVE ALPHABET IS IN OTHER THAN ROMAN LETTERS. WRITE YOUR NAME IN NATURALIZATION ADJUSTMENT OF STATUS YOUR NATIVE ALPHABET IN THIS SPACE: OTHER (SPECIFY): PENALTIES: SEVERE PENALTIES ARE PROVIDED BY LAW FOR KNOWINGLY AND WILLFULLY FALSIFYING 11/6/68 OR CONCEALING A MATERIAL FACT. DATE Wing (SIGNATURE Ben OF APPLICANT OR Dong PETITIONER) COMPLETE THIS BOX (FAMILY NAME) (GIVEN NAME) (MIDDLE NAME) (ALIEN REGISTRATION NUMBER) WONG Ben Dong A12 250 430 (OTHER AGENCY USE) (INS USE) (daughter REFER no DATA in KE HUANG law (nee CH'EN) GEN-hsia LOS WHICH WAS SENT THE Los angeles (All-94-27a) etc. T/C - 249 11/10/69 (DATE) OFFICE OF INS ON local basis investigative fi : To check FB1 this reply is result of check of FILL records, request arrest to Identification Divident Dre submitted Water are necessary for positive Finger dis, 2) Rec. Br. FORM G-325A
""")

{'REVISION_DATE': ['REV. 4-1-67'],
 'OMB_NUMBER': ['43-R436'],
 'COUNTRY': ['UNITED STATES', 'CANADA', 'CHINA'],
 'NATIONALITY': ['CHINESE'],
 'STATE_CITY': ['CALIF.', 'LOS ANGELES', 'U.S.A.', 'CANTON', 'Los angeles'],
 'EMPLOYER': ['SELF-EMPLOYED'],
 'FORM_NUMBER': ['G-325A']}

## Using Classification Model to predict classes from an Image

The 4 classes currently supported are:
1. Documents (Papers that follow specific formatting and prints)
2. Letter (Papers with large blocks of texts)
3. Photographs
4. Miscellaneous

In [1]:
from classification_models.image.load_classifier import Pretrained_Image_Classifier


In [3]:
model_class = Pretrained_Image_Classifier("linear_layer.pth",device='cpu')


Loading a pretrained model + linear head


Using cache found in /Users/ajay/.cache/torch/hub/facebookresearch_dinov2_main


In [4]:
output, features = model_class.inference("https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A10001331_0001/full/full/0/default.jpg")

In [5]:
output.shape, features.shape

(torch.Size([1, 4]), torch.Size([1, 1536]))

Taking the most confident class

In [6]:
import torch
torch.argmax(output)

tensor(0)

## Test out certificate of Naturalization 

In [2]:
from misc_techniques import detect_cert_nat

In [12]:
cert_nat_image, cert_nat_text = detect_cert_nat.load_template_embeddings()

### If Embeddings are present

For my data, I precomputed the embeddings from an embedding extractor and stored it separately to save time on experimentation. 

You could either do the same and do the embedding calculation online or load the embeddings from a pickle file after precomputing them.

In [13]:
image_embeddings = detect_cert_nat.init_image_embeddings("/Users/ajay/Documents/Oncampus/TrainingFramework/Newer/Model_Trainer/data/parallel_results.pkl")
text_embeddings = detect_cert_nat.init_text_embeddings("/Users/ajay/Documents/Oncampus/TrainingFramework/text_embeddings_textract.pkl")

In [15]:
i = 3 
image_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([i], cert_nat_image, image_embeddings, mode='pkl')
text_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([i], cert_nat_text, text_embeddings, mode='pkl')

detect_cert_nat.verify_cert_nat(text_cosine_sim, image_cosine_sim)

1

### If Embeddings aren't Present

In [14]:
piece_of_text = """'Copied at the National Archives at Kansas City DUPLICATE TO BE FORWARDED TO No. 8050754 IMMIGRATION AND INATURALIZATION SERVICE Pelition no 17603 Personal description, of holder as of Date of birth April 23. 1878 sex M ; complexion fair colorsheries grey color of hair grey height 5 fees 7 inches; weight 163 pounds; visible distinctive marks wart corner left eye Maritalstatus married ed to less formerly Russia I certify that the description, above given is have and that a likeness of me. Sun sign Wladmins Fedowosth Love Complete and truce signature of holder) State of New York } SS: County of Albany Beitknown, that at a termofthe Supreme Court of New York, Albany County held pursuantly, lawal Albany, New York on June 15, 1961 the Court havingfoundshal Wladimir Fedorovich Livin then residing at 22 Forest Dr Albany New York inlends loveside permanentlyin the United States when sorrequired by the naturalization Laws of he United. States), had in callotherrespects, complied with the applicable provisions of such nahuralizeation laws, and was entilled to be Seal admilled docilizenship thereupon ordered that such person be and(s)hewas admilled as a cilizer of the United States of America. Indestimony whereofthesealofthe courtis hereunto, officed this 15th day of June in the year of own Lord nineteen hundred and Sixty-one and of our. Independence the one hundred and Eighty-fifth Breadd L. Linch punishable Insuriolation as such) to copypain., of the u. S. Code photograph. land Clerk of the Supreme Court. or otherwise illegally use this certificate. By Francle missuff Deputy Clerk.'
"""
url = 'https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A10001331_0003/full/full/0/default.jpg'



In [16]:
from classification_models.image.load_classifier import Pretrained_Image_Classifier
image_model = Pretrained_Image_Classifier("linear_layer.pth")
text_model = detect_cert_nat.init_text_model()


Metal Performance Shaders (MPS) is available on mac
Loading a pretrained model + linear head


Using cache found in /Users/ajay/.cache/torch/hub/facebookresearch_dinov2_main
/Users/ajay/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/ajay/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/ajay/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [19]:
image_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([url], cert_nat_image, None, image_model, mode='image_model')
text_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([piece_of_text], cert_nat_text, None, text_model, mode='text_model')

detect_cert_nat.verify_cert_nat(text_cosine_sim, image_cosine_sim)

1

## Detecting if a certain text is a form G-325A

In [7]:
from misc_techniques.detect_G325A import detect_g325av5 

In [8]:
text = """Screened by NARA, 9/20/2022 NOTICE TO PERSONS FILING FOR SPOUSES IF MARRIED LESS THAN TWO YEARS Pursuant to section 216 of the Immigration and Nationality Act, your alien spouse may be granted conditional permanent resident status in the United States as of the date he or she is admitted or adjusted to conditional status by an officer of the Immigration and Naturalization Service. Both you and your conditional permanent resident spouse are required to file a petition, Form I-751, Joint Petition to Remove Conditional Basis of Alien's Permanent Resident Status, during the ninety day period immediately before the second anniversary of the date your alien spouse was granted conditional permanent residence. Otherwise, the rights, privileges, responsibilities and duties which apply to all other permanent residents apply equally to a conditional permanent resident. A conditional permanent resident is not limited to the right to apply for naturalization, to file petitions in behalf of qualifying relatives, or to reside permanently in the United States as an immigrant in accordance with the immigration laws. Failure to file Form I-751, Joint Petition to Remove the Conditional Basis of Alien's Permanent Resident Status, will result in termination of permanent residence status and initiation of deportation proceedings. NOTE: You must complete Items 1 through 6 to assure that petition approval is recorded. Do not write in the section below item 6. 1. Name of relative (Family name in CAPS) (First) (Middle) CHECKLIST ISNARDY MARGUERITE 2. Other names used by relative (Including maiden name) Have you answered each ISNARDY MARGUERITE question? 3. Country of relative's birth 4. Date of relative's birth (Month/Day/Year) Port Au Prince Haiti 05/06/ 1905 Have you signed the petition? Have you enclosed: 5. Your name (Last name in CAPS) (First) (Middle) HONORE ROSANA (617 17-282-8630 282 Your phone number * The filing fee for each SECTION DATE PETITION FILED petition? Action Stamp 201 (b)(spouse) Proof of your citizenship or 201 (b)(child) lawful permanent residence? XX (b)(parent) All required supporting 203 (a)(1) STATESIDE documents for each petition? 203 (a)(2) CRITERIA GRANTED 203 (a)(4) If you are filing for your 203 (a)(5) SENT TO CONSUL AT; husband or wife have you included: Your picture? His or herpicture? Your G-325A? His or her G-325A? Relative Petition Card Form I-130 (Rev. 10/13/98)N"""

detect_g325av5(text)


0

In [9]:
text = """U.S. Department of Justice FORM G-325A OMB No. 1115-0066 Immigration and Naturalization Service BIOGRAPHIC INFORMATION Approval expires 4-30-85 (Family name) (First name) (Middle name) MALE BIRTHDATE(Mo.-Day-Yr.) NATIONALITY / FILE NUMBER De IA HOYA Vicente SANches FEMALE 5-12-12 MEXICANA 410-534-184 ALL OTHER NAMES USED (Including names by previous marriages) CITY AND COUNTRY OF BIRTH SOCIAL SECURITY NO. DuraNGO MEXICO (If any) 457-40-4959 FAMILY NAME FIRST NAME DATE, CITY AND COUNTRY OF BIRTH(If known) CITY AND COUNTRY OF RESIDENCE FATHER De IA HOYA ENrique Durango MEXICO (D) MOTHER (Maiden name) SANChe's CONSTANZA Durango MEXICO ( D ) HUSBAND (If none, so state) FAMILY NAME FIRST NAME BIRTHDATE CITY & COUNTRY OF BIRTH DATE OF MARRIAGE PLACE OF MARRIAGE OR (For wife, give maiden name) WIFE De , IA HOYA Torres DoLores 3-15-40 NAYAVIT MEX Lurs-B.S 10-26-72 SONOTA FORMER HUSBANDS OR WIVES(if none,s state) FAMILY NAME (For wife, give maiden name) FIRST NAME BIRTHDATE DATE & PLACE OF MARRIAGE DATE AND PLACE OF TERMINATION OF MARRIAGE SALAS GuAdeLupe 12-8-20 Duranco MEX 7-15-60 APPLICANT'S RESIDENCE LAST FIVE YEARS. LIST PRESENT ADDRESS FIRST. FROM TO STREET AND NUMBER CITY PROVINCE OR STATE COUNTRY MONTH YEAR MONTH YEAR ChihuahuA #139 MEXICAL B- CALIF MEXICO 3 65 PRESENT TIME APPLICANT'S LAST ADDRESS OUTSIDE THE UNITED STATES OF MORE THAN ONE YEAR FROM TO STREET AND NUMBER CITY PROVINCE OR STATE COUNTRY MONTH YEAR MONTH YEAR APPLICANT'S EMPLOYMENT LAST FIVE YEARS. (IF NONE, so STATE.) LIST PRESENT EMPLOYMENT FIRST FROM TO FULL NAME AND ADDRESS OF EMPLOYER OCCUPATION (SPECIFY) MONTH YEAR MONTH YEAR E. and L. AVILA Trucking LAbor 6 83 PRESENT TIME Browley CAL. watch MEN 8-15 85 Show below last occupation abroad if not shown above. (Include all information requested above.) THIS FORM IS SUBMITTED IN CONNECTION WITH APPLICATION FOR: SIGNATURE OF APPLICANT DATE NATURALIZATION STATUS AS PERMANENT RESIDENT OTHER (SPECIFY): Vicente dela Hoyan S. 3-31-88 IF YOUR NATIVE ALPHABET is IN OTHER THAN ROMAN LETTERS WRITE YOUR NAME IN YOUR NATIVE ALPHABET IN THIS SPACE Are all copies legible? Yes PENALTIES: SEVERE PENALTIES ARE PROVIDED BY LAW FOR KNOWINGLY AND WILLFULLY FALSIFYING OR CONCEALING A MATERIAL FACT. APPLICANT: BE SURE TO PUT YOUR NAME AND ALIEN REGISTRATION NUMBER IN THE BOX OUTLINED BY HEAVY BORDER BELOW. COMPLETE THIS BOX (Family name) (Given name) (Middle name) (Alien registration number) De LA HOYA ViceNTe SANChes A10-534-184 OFFICE: SND N-400 Form G-325 A (Rev. 10-1-82) DATE: (1) IDENT. MAY 03 1988""" 
detect_g325av5(text)


1

## Year, Date (primarily) Extractor

In [10]:
from misc_techniques import flair_extract

2024-07-16 19:34:06,406 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [11]:
text = """Screened by NARA, 9/20/2022 NOTICE TO PERSONS FILING FOR SPOUSES IF MARRIED LESS THAN TWO YEARS Pursuant to section 216 of the Immigration and Nationality Act, your alien spouse may be granted conditional permanent resident status in the United States as of the date he or she is admitted or adjusted to conditional status by an officer of the Immigration and Naturalization Service. Both you and your conditional permanent resident spouse are required to file a petition, Form I-751, Joint Petition to Remove Conditional Basis of Alien's Permanent Resident Status, during the ninety day period immediately before the second anniversary of the date your alien spouse was granted conditional permanent residence. Otherwise, the rights, privileges, responsibilities and duties which apply to all other permanent residents apply equally to a conditional permanent resident. A conditional permanent resident is not limited to the right to apply for naturalization, to file petitions in behalf of qualifying relatives, or to reside permanently in the United States as an immigrant in accordance with the immigration laws. Failure to file Form I-751, Joint Petition to Remove the Conditional Basis of Alien's Permanent Resident Status, will result in termination of permanent residence status and initiation of deportation proceedings. NOTE: You must complete Items 1 through 6 to assure that petition approval is recorded. Do not write in the section below item 6. 1. Name of relative (Family name in CAPS) (First) (Middle) CHECKLIST ISNARDY MARGUERITE 2. Other names used by relative (Including maiden name) Have you answered each ISNARDY MARGUERITE question? 3. Country of relative's birth 4. Date of relative's birth (Month/Day/Year) Port Au Prince Haiti 05/06/ 1905 Have you signed the petition? Have you enclosed: 5. Your name (Last name in CAPS) (First) (Middle) HONORE ROSANA (617 17-282-8630 282 Your phone number * The filing fee for each SECTION DATE PETITION FILED petition? Action Stamp 201 (b)(spouse) Proof of your citizenship or 201 (b)(child) lawful permanent residence? XX (b)(parent) All required supporting 203 (a)(1) STATESIDE documents for each petition? 203 (a)(2) CRITERIA GRANTED 203 (a)(4) If you are filing for your 203 (a)(5) SENT TO CONSUL AT; husband or wife have you included: Your picture? His or herpicture? Your G-325A? His or her G-325A? Relative Petition Card Form I-130 (Rev. 10/13/98)N"""

output = flair_extract.extract_country_years(text)

In [12]:
for k,v in output.items():
    print(k,"-----------------")
    print(v)

dates -----------------
['9/20/2022', '10/13/98']
cardinals -----------------
['216', '1', '6', '6.', '2', '4', '5', '201', '201', 'XX', '203', '1', '203', '2', '203', '4', '203', '5']
places -----------------
['the United States', 'the United States', 'Port Au Prince', 'Haiti']
countries -----------------
['Haiti', 'United States']
years -----------------
['2022', '1905']


In [4]:
text = """U.S. Department of Justice FORM G-325A OMB No. 1115-0066 Immigration and Naturalization Service BIOGRAPHIC INFORMATION Approval expires 4-30-85 (Family name) (First name) (Middle name) MALE BIRTHDATE(Mo.-Day-Yr.) NATIONALITY / FILE NUMBER De IA HOYA Vicente SANches FEMALE 5-12-12 MEXICANA 410-534-184 ALL OTHER NAMES USED (Including names by previous marriages) CITY AND COUNTRY OF BIRTH SOCIAL SECURITY NO. DuraNGO MEXICO (If any) 457-40-4959 FAMILY NAME FIRST NAME DATE, CITY AND COUNTRY OF BIRTH(If known) CITY AND COUNTRY OF RESIDENCE FATHER De IA HOYA ENrique Durango MEXICO (D) MOTHER (Maiden name) SANChe's CONSTANZA Durango MEXICO ( D ) HUSBAND (If none, so state) FAMILY NAME FIRST NAME BIRTHDATE CITY & COUNTRY OF BIRTH DATE OF MARRIAGE PLACE OF MARRIAGE OR (For wife, give maiden name) WIFE De , IA HOYA Torres DoLores 3-15-40 NAYAVIT MEX Lurs-B.S 10-26-72 SONOTA FORMER HUSBANDS OR WIVES(if none,s state) FAMILY NAME (For wife, give maiden name) FIRST NAME BIRTHDATE DATE & PLACE OF MARRIAGE DATE AND PLACE OF TERMINATION OF MARRIAGE SALAS GuAdeLupe 12-8-20 Duranco MEX 7-15-60 APPLICANT'S RESIDENCE LAST FIVE YEARS. LIST PRESENT ADDRESS FIRST. FROM TO STREET AND NUMBER CITY PROVINCE OR STATE COUNTRY MONTH YEAR MONTH YEAR ChihuahuA #139 MEXICAL B- CALIF MEXICO 3 65 PRESENT TIME APPLICANT'S LAST ADDRESS OUTSIDE THE UNITED STATES OF MORE THAN ONE YEAR FROM TO STREET AND NUMBER CITY PROVINCE OR STATE COUNTRY MONTH YEAR MONTH YEAR APPLICANT'S EMPLOYMENT LAST FIVE YEARS. (IF NONE, so STATE.) LIST PRESENT EMPLOYMENT FIRST FROM TO FULL NAME AND ADDRESS OF EMPLOYER OCCUPATION (SPECIFY) MONTH YEAR MONTH YEAR E. and L. AVILA Trucking LAbor 6 83 PRESENT TIME Browley CAL. watch MEN 8-15 85 Show below last occupation abroad if not shown above. (Include all information requested above.) THIS FORM IS SUBMITTED IN CONNECTION WITH APPLICATION FOR: SIGNATURE OF APPLICANT DATE NATURALIZATION STATUS AS PERMANENT RESIDENT OTHER (SPECIFY): Vicente dela Hoyan S. 3-31-88 IF YOUR NATIVE ALPHABET is IN OTHER THAN ROMAN LETTERS WRITE YOUR NAME IN YOUR NATIVE ALPHABET IN THIS SPACE Are all copies legible? Yes PENALTIES: SEVERE PENALTIES ARE PROVIDED BY LAW FOR KNOWINGLY AND WILLFULLY FALSIFYING OR CONCEALING A MATERIAL FACT. APPLICANT: BE SURE TO PUT YOUR NAME AND ALIEN REGISTRATION NUMBER IN THE BOX OUTLINED BY HEAVY BORDER BELOW. COMPLETE THIS BOX (Family name) (Given name) (Middle name) (Alien registration number) De LA HOYA ViceNTe SANChes A10-534-184 OFFICE: SND N-400 Form G-325 A (Rev. 10-1-82) DATE: (1) IDENT. MAY 03 1988""" 
output = flair_extract.extract_country_years(text)

for k,v in output.items():
    print(k,"-----------------")
    print(v)

printing dates ['LAST FIVE YEARS', 'MONTH YEAR', 'MONTH YEAR', 'MORE THAN ONE YEAR', 'MONTH YEAR', 'MONTH YEAR', 'FIVE YEARS', 'MONTH YEAR', 'MONTH YEAR', 'MAY 03 1988']
dates -----------------
[]
cardinals -----------------
['4-30-85', '3-15-40', '10-26-72', '12-8-20', '7-15-60', '139', '3', '65', '83', '10-1-82', '1']
places -----------------
['DuraNGO', 'MEXICO', 'Durango', 'MEXICO', 'Durango', 'MEXICO', 'Duranco', 'MEXICO', 'THE UNITED STATES']
countries -----------------
['Mexico', 'United States']
years -----------------
['1988']


## Inference on a csv

In [ ]:
import json 
import torch
from tqdm import tqdm
# from metadata_extraction.run_llm_pipeline import Extraction
# from classification_models.image.load_classifier import Pretrained_Image_Classifier
# import sys
# module_path = "/scratch/ag8172/Deployment_Framework/Misc_Techniques"
# sys.path.append(module_path)
from Misc_Techniques import detect_cert_nat
from Misc_Techniques.detect_G325A import detect_g325av5 
from Misc_Techniques import flair_extract
from metadata_extraction.run_llm_pipeline import Extraction
from classification_models.image.load_classifier import Pretrained_Image_Classifier
import pandas as pd 
import os

df = pd.read_csv("/scratch/combined_csv.csv") # dummy directory
# os.mkdir('/Users/ajay/Documents/Oncampus/TrainingFramework/metadata_outputs/') # make directory in scratch

cert_nat_image, cert_nat_text = detect_cert_nat.load_template_embeddings()
image_embeddings = detect_cert_nat.init_image_embeddings("/scratch/parallel_results.pkl")
text_embeddings = detect_cert_nat.init_text_embeddings("/scratch/text_embeddings_textract.pkl")

model_class = Pretrained_Image_Classifier("linear_layer.pth")
ext_class = Extraction("./metadata_extraction/configs/fewshot_v2.cfg", "./metadata_extraction/configs/examples_g325a_2.yml")

model_inf_map = {0:'form',1:'letter',2:'photograph',3:'misc'}


''' 
model_inf_map = {0:'form',1:'letter',2:'photo',3:'misc'}
for each url:
        run inference, get class
        {'document_type': model_inf_map[output]}
        if doc not photograph or misc:
                get text       
                check certnat
                if certnat:
                        {'is_cert_nat':1}
                elif g325a:
                        {'is_g325a':True}
                        invoke llm
                        {'llm_<attribute>':}

                run generalextraction with text
                {<attribute>}
                
'''
error_log_path = "/scratch/error_log.txt"
error_noocr_path = "/scratch/no_ocr_log.txt"
for i in tqdm(df.index.tolist()[24028:]):
        try:
                document_wise_result = dict()
                url = df.iloc[i]['url']
                filename = str(i) + "_" + "_".join(df.iloc[i]['url'].split('/')[5].split('_')[1:]) + ".json"
                filename = os.path.join("/scratch/metadata_outputs/", filename)
                output, features = model_class.inference(url)
                document_wise_result['document_type'] = model_inf_map[torch.argmax(output).item()]
                document_wise_result['is_g325a'] = False
                document_wise_result['is_cert_naturalization'] = False 
                if document_wise_result['document_type'] not in ['photograph','misc']:
                        # get text
                        # text was concatenated strings obtained from aws textract
                        text = df.iloc[i]['Detected Text']
                        if not isinstance(text,float):
                                
                                image_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([i], cert_nat_image, image_embeddings, mode='pkl')
                                text_cosine_sim = detect_cert_nat.compute_cosine_similarity_scores_from_pkls([i], cert_nat_text, text_embeddings, mode='pkl')
                                is_cert_nat = detect_cert_nat.verify_cert_nat(text_cosine_sim, image_cosine_sim)
                                if is_cert_nat:
                                        document_wise_result['is_cert_naturalization'] = True
                                else:
                                        document_wise_result['is_cert_naturalization'] = False 
                                
                                is_g325a = detect_g325av5(text)
                                document_wise_result['g325a_attributes'] = dict()
                                if is_g325a:
                                        document_wise_result['is_g325a'] = True
                                        output = ext_class.extract_metadata(f"""{text}""")
                                        document_wise_result['g325a_attributes'] = dict()
                                        for k,v in output.items():
                                                document_wise_result['g325a_attributes']['LLM_'+k] = v

                                else:
                                        document_wise_result['is_g325a'] = False
                                
                                output = flair_extract.extract_country_years(text)
                                document_wise_result['countries'] = output['countries']
                                document_wise_result['years'] = output['years']
                        else:
                                with open(error_noocr_path, "a") as error_log_file:
                                        error_log_file.write(f"{i}, OCR Content Absent\n")
                with open(filename, "w") as json_file:
                        json.dump(document_wise_result, json_file)
        except Exception as e:
                with open(error_log_path, "a") as error_log_file:
                        error_log_file.write(f"{i}, {str(e)}\n")
                print("Exception occurred for", i, e)

        

